In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarusr
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser
from scipy.stats import entropy

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Wed Aug  3 00:44:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 68%   80C    P2   162W / 260W |   7626MiB / 11019MiB |     99%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_us = db.lda_topic_us
    
    
except errors.ConnectionFailure as err:
    print(err)

In [5]:
db_us

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'lda_topic_us')

# US

In [6]:
us_topic = db_us.aggregate([
        {"$unwind" : "$classification"}
])


In [7]:
us_df = pd.DataFrame(us_topic)

In [8]:
# 取cpc前3碼
us_df["cpc_n"] = us_df.classification.apply(lambda x: x[0:3])

In [9]:
us_df

,_id,patentNumber,appDate,company,classification,brief,topic,cpc_n
0,62c6c7201ef3ba661de7d8f3,US20130223235A1,2010-07-14 04:00:00,[BlackBerry Ltd],H04W36/30,A UE comprising a processor configured to perf...,18,H04
1,62c6c7201ef3ba661de7d8f3,US20130223235A1,2010-07-14 04:00:00,[BlackBerry Ltd],H04W48/20,A UE comprising a processor configured to perf...,18,H04
2,62c6c7201ef3ba661de7d8f3,US20130223235A1,2010-07-14 04:00:00,[BlackBerry Ltd],H04W36/04,A UE comprising a processor configured to perf...,18,H04
3,62c6c7201ef3ba661de7d8f3,US20130223235A1,2010-07-14 04:00:00,[BlackBerry Ltd],H04W84/045,A UE comprising a processor configured to perf...,18,H04
4,62c6c7201ef3ba661de7d8f4,US8458315B2,2010-12-06 05:00:00,[SAP SE],H04W4/00,Embodiments of the present invention include s...,16,H04
...,...,...,...,...,...,...,...,...
369993,62c6c7201ef3ba661de86fee,US20200344025A1,2020-07-08 04:00:00,[Fujitsu Ltd],H04W72/0406,A wireless communication system includes: a ba...,1,H04
369994,62c6c7201ef3ba661de86fee,US20200344025A1,2020-07-08 04:00:00,[Fujitsu Ltd],H04W72/042,A wireless communication system includes: a ba...,1,H04
369995,62c6c7201ef3ba661de86fee,US20200344025A1,2020-07-08 04:00:00,[Fujitsu Ltd],H04W72/14,A wireless communication system includes: a ba...,1,H04
369996,62c6c7201ef3ba661de86fee,US20200344025A1,2020-07-08 04:00:00,[Fujitsu Ltd],H04W76/27,A wireless communication system includes: a ba...,1,H04


In [10]:
us_df2 = us_df.groupby(["cpc_n","topic"],as_index=False).size()

In [11]:
us_df2

,cpc_n,topic,size
0,A01,0,16
1,A01,1,84
2,A01,2,15
3,A01,3,18
4,A01,4,6
...,...,...,...
1367,Y10,13,4
1368,Y10,14,4
1369,Y10,17,2
1370,Y10,19,1


## 找出前10的CPC計算各類別的topic比例

In [12]:
us_cpc_ls = ["H04","G06","H01","G01","A61","H03","B60","G08","Y02","G05"]

In [13]:
us_cpc_top10 = us_df2[us_df2.cpc_n.isin(us_cpc_ls)]
us_cpc_top10

,cpc_n,topic,size
134,A61,0,638
135,A61,1,1261
136,A61,2,537
137,A61,3,417
138,A61,4,214
...,...,...,...
1329,Y02,16,192
1330,Y02,17,159
1331,Y02,18,81
1332,Y02,19,120


In [14]:
us_cpc_sum = us_cpc_top10.groupby(["cpc_n"],as_index=False).sum()[["cpc_n","size"]]
us_cpc_sum.rename(columns = {'size':'sum'}, inplace = True)
us_cpc_sum

,cpc_n,sum
0,A61,8227
1,B60,4742
2,G01,7853
3,G05,4309
4,G06,75971
5,G08,4650
6,H01,15964
7,H03,5040
8,H04,201297
9,Y02,4320


In [15]:
sum(us_cpc_sum["sum"])

332373

In [16]:
us_cpc_top10_fin = pd.merge(us_cpc_top10, us_cpc_sum, on="cpc_n")
us_cpc_top10_fin

,cpc_n,topic,size,sum
0,A61,0,638,8227
1,A61,1,1261,8227
2,A61,2,537,8227
3,A61,3,417,8227
4,A61,4,214,8227
...,...,...,...,...
205,Y02,16,192,4320
206,Y02,17,159,4320
207,Y02,18,81,4320
208,Y02,19,120,4320


In [17]:
# 計算topic比例
us_cpc_top10_fin["topic_ratio"] = us_cpc_top10_fin.apply(lambda x: x["size"]/x["sum"],axis=1)

In [18]:
us_cpc_top10_fin

,cpc_n,topic,size,sum,topic_ratio
0,A61,0,638,8227,0.077550
1,A61,1,1261,8227,0.153276
2,A61,2,537,8227,0.065273
3,A61,3,417,8227,0.050687
4,A61,4,214,8227,0.026012
...,...,...,...,...,...
205,Y02,16,192,4320,0.044444
206,Y02,17,159,4320,0.036806
207,Y02,18,81,4320,0.018750
208,Y02,19,120,4320,0.027778


## 計算Entropy
計算各CPC底下topic的混亂程度

In [19]:
entropy(list(us_cpc_top10_fin[us_cpc_top10_fin.cpc_n=="H04"]["topic_ratio"]), base=2)

4.083148897901255

In [20]:
us_entropy_ls = []
for i in us_cpc_ls:
    en_num = entropy(list(us_cpc_top10_fin[us_cpc_top10_fin.cpc_n==i]["topic_ratio"]), base=2)
    us_entropy_ls.append(en_num)

In [21]:
us_entropy_ls

[4.083148897901255,
 4.193266282113336,
 4.239901510051913,
 4.160411453979962,
 4.1699865767528,
 4.10210193621254,
 4.049118682601658,
 4.162609594405477,
 4.111090508351771,
 4.176232249742832]

In [22]:
us_entropy = pd.DataFrame((zip(us_cpc_ls, us_entropy_ls)), columns = ['cpc', 'entropy'])
us_entropy

,cpc,entropy
0,H04,4.083149
1,G06,4.193266
2,H01,4.239902
3,G01,4.160411
4,A61,4.169987
5,H03,4.102102
6,B60,4.049119
7,G08,4.162610
8,Y02,4.111091
9,G05,4.176232
